In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# --------------------------
# 1. Leer coordenadas originales
# --------------------------
df_raw = pd.read_csv("../Data/DIM_TIENDA.csv")
geometry = [Point(xy) for xy in zip(df_raw['LONGITUD_NUM'], df_raw['LATITUD_NUM'])]
gdf_coords = gpd.GeoDataFrame(df_raw, geometry=geometry, crs="EPSG:4326")

# --------------------------
# 2. Leer y unir AGEBs de NL y TAMPS
# --------------------------
gdf_nl = gpd.read_file("../Data/Datos-Inegi/ageb/19_nuevoleon/conjunto_de_datos/19a.shp").to_crs("EPSG:4326")
gdf_tamps = gpd.read_file("../Data/Datos-Inegi/ageb/28_tamaulipas/conjunto_de_datos/28a.shp").to_crs("EPSG:4326")

if 'AMBITO' in gdf_nl.columns:
    gdf_nl = gdf_nl[gdf_nl['AMBITO'] == '1']
if 'AMBITO' in gdf_tamps.columns:
    gdf_tamps = gdf_tamps[gdf_tamps['AMBITO'] == '1']

gdf_agebs = pd.concat([gdf_nl, gdf_tamps], ignore_index=True)

# --------------------------
# 3. Spatial join AGEB
# --------------------------
gdf_result = gpd.sjoin(gdf_coords, gdf_agebs, how='left', predicate='within')
gdf_result['ENTIDAD'] = gdf_result['CVE_ENT'].astype(str).str.zfill(2)
gdf_result['MUN'] = gdf_result['CVE_MUN'].astype(str).str.zfill(3)
gdf_result['AGEB'] = gdf_result['CVE_AGEB'].astype(str).str.zfill(4)

# Diagnóstico: tiendas fuera de AGEBs
sin_ageb = gdf_result[gdf_result['CVE_AGEB'].isna()]
print(f"❌ Tiendas fuera de cualquier AGEB: {len(sin_ageb)}")
# print all 'TIENDA_ID' in a list without 'sin_ageb'
print("🗺️ Tiendas fuera de AGEBs:"
        f" {sin_ageb['TIENDA_ID'].unique().tolist()}")


# --------------------------
# 4. Localidades más cercanas
# --------------------------
gdf_loc_nl = gpd.read_file("../Data/Datos-Inegi/ageb/19_nuevoleon/conjunto_de_datos/19l.shp").to_crs("EPSG:4326")
gdf_loc_tamps = gpd.read_file("../Data/Datos-Inegi/ageb/28_tamaulipas/conjunto_de_datos/28l.shp").to_crs("EPSG:4326")
gdf_localidades = pd.concat([gdf_loc_nl, gdf_loc_tamps], ignore_index=True)

if gdf_localidades.geometry.is_empty.all():
    gdf_localidades['geometry'] = gdf_localidades.apply(lambda row: Point(row['LONGITUD'], row['LATITUD']), axis=1)

gdf_localidades_m = gdf_localidades.to_crs(epsg=6372)
gdf_result_m = gdf_result.to_crs(epsg=6372)

def encontrar_localidad_mas_cercana(punto):
    return gdf_localidades_m.distance(punto).idxmin()

gdf_result['LOC'] = gdf_result_m.geometry.apply(
    lambda p: gdf_localidades.loc[encontrar_localidad_mas_cercana(p), 'CVE_LOC']
).astype(str).str.zfill(4)

# --------------------------
# 5. Crear df_tiendas con CLAVE_AGEB
# --------------------------
columnas_originales = [
    'TIENDA_ID', 'PLAZA_CVE', 'NIVELSOCIOECONOMICO_DES', 'ENTORNO_DES',
    'MTS2VENTAS_NUM', 'PUERTASREFRIG_NUM', 'CAJONESESTACIONAMIENTO_NUM',
    'LATITUD_NUM', 'LONGITUD_NUM', 'SEGMENTO_MAESTRO_DESC'
]

df_tiendas = gdf_result[columnas_originales + ['ENTIDAD', 'MUN', 'LOC', 'AGEB']].copy()

for col, width in zip(['ENTIDAD', 'MUN', 'LOC', 'AGEB'], [2, 3, 4, 4]):
    df_tiendas[col] = df_tiendas[col].astype(str).str.zfill(width)

df_tiendas['CLAVE_AGEB'] = df_tiendas['ENTIDAD'] + df_tiendas['MUN'] + df_tiendas['LOC'] + df_tiendas['AGEB']
# df_tiendas.to_csv("../Data/tiendas_con_claves.csv", index=False)

# --------------------------
# 6. Leer y preparar datos demográficos
# --------------------------
df_nl = pd.read_csv("../Data/municipios_raw/RESAGEBURB_19CSV20.csv", dtype=str, low_memory=False)
df_tamps = pd.read_csv("../Data/municipios_raw/RESAGEBURB_28CSV20.csv", dtype=str, low_memory=False)
df_demog = pd.concat([df_nl, df_tamps], ignore_index=True)

for col, width in zip(['ENTIDAD', 'MUN', 'LOC', 'AGEB'], [2, 3, 4, 4]):
    df_demog[col] = df_demog[col].astype(str).str.zfill(width)

df_demog['CLAVE_AGEB'] = df_demog['ENTIDAD'] + df_demog['MUN'] + df_demog['LOC'] + df_demog['AGEB']
df_demog.to_csv("../Data/municipios.csv", index=False)

# --------------------------
# 7. Merge con datos demográficos
# --------------------------
df_final = df_tiendas.merge(df_demog, on='CLAVE_AGEB', how='left')
print("🧮 Total tiendas:", df_tiendas['TIENDA_ID'].nunique())
print("📌 Total claves en df_tiendas:", df_tiendas['CLAVE_AGEB'].nunique())
print("📌 Total claves en df_demog:", df_demog['CLAVE_AGEB'].nunique())
print("✅ Tiendas con datos demográficos:", df_final['TIENDA_ID'].nunique())
print("🚫 Tiendas sin datos demográficos:", df_final['POBTOT'].isna().sum())


# Diagnóstico de claves
df_con_nan = df_tiendas[df_tiendas['CLAVE_AGEB'].str.contains("nan", case=False, na=False)]
print(f"⚠️ Claves mal formadas: {len(df_con_nan)}")

# --------------------------
# 8. Exportar tiendas válidas
# --------------------------
df_validas = df_final.copy()
# df_validas.to_csv("../Data/tiendas_con_demografia.csv", index=False)

# --------------------------
# 9. Agrupación por tienda con variables seleccionadas
# --------------------------
variables_demograficas = [
    'POBTOT', 'TOTHOG', 'POCUPADA', 'PDESOCUP',
    'P_18A24', 'P_18A24_F', 'P_18A24_M', 'VPH_REFRI',
    'VPH_AUTOM', 'VPH_PC', 'VPH_NDACMM', 'VPH_SINCINT',
    'P_12YMAS', 'P_15YMAS', 'P_15A49_F', 'POB15_64',
    'PEA', 'PE_INAC', 'P12YM_CASA', 'P12YM_SOLT',
    'P15YM_AN', 'P15YM_SE', 'P15PRI_CO', 'P15SEC_CO',
    'P18YM_PB'
]

# Asegurarnos de que estén en el DataFrame
variables_demograficas = [col for col in variables_demograficas if col in df_validas.columns]

# Limpieza de datos demográficos
df_validas[variables_demograficas] = (
    df_validas[variables_demograficas]
    .replace('*', 0)
    .fillna(0)
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
)

from collections import Counter

def moda_no_nula(x):
    x_no_na = x.dropna()
    return x_no_na.mode().iloc[0] if not x_no_na.empty else None

# Agrupación final
columnas_agrupacion = [col for col in columnas_originales if col != 'TIENDA_ID']
# Diagnóstico: verificar consistencia de columnas originales por tienda
inconsistencias = df_validas.groupby('TIENDA_ID')[columnas_agrupacion].nunique()
inconsistentes = inconsistencias[inconsistencias.gt(1).any(axis=1)]
print(f"⚠️ Tiendas con valores distintos en columnas originales: {len(inconsistentes)}")
if not inconsistentes.empty:
    print(inconsistentes.head())
df_grouped = df_validas.groupby('TIENDA_ID').agg({
    **{col: moda_no_nula for col in columnas_agrupacion},
    **{col: 'sum' for col in variables_demograficas}
}).reset_index()

# Exportar df_grouped
df_grouped.to_csv("../Data/tiendas_ageb_demografia.csv", index=False)
print("✅ 'tiendas_ageb_demografia.csv' generado correctamente.")

# --------------------------
# 10. Incorporar VENTA mensual
# --------------------------
# Cargar IDs a eliminar
tiendas_test = pd.read_csv("../Data/DIM_TIENDA_TEST.csv")['TIENDA_ID'].unique()

# Cargar ventas y filtrar
df_ventas = pd.read_csv("../Data/venta.csv")
df_ventas = df_ventas[~df_ventas['TIENDA_ID'].isin(tiendas_test)]
df_ventas['MES_ID'] = df_ventas['MES_ID'].astype(str)

print("📊 Ventas únicas (TIENDA_ID):", df_ventas['TIENDA_ID'].nunique())
print("📦 Tiendas agregadas en df_grouped:", df_grouped['TIENDA_ID'].nunique())

# Unir ventas por TIENDA_ID
df_final_ventas = df_ventas.merge(df_grouped, on='TIENDA_ID', how='left')
df_final_ventas.to_csv("../Data/tiendas_ageb_demografia_ventas.csv", index=False)
print("✅ 'tiendas_ageb_demografia_ventas.csv' generado correctamente.")


❌ Tiendas fuera de cualquier AGEB: 35
🗺️ Tiendas fuera de AGEBs: [904, 961, 926, 927, 858, 932, 939, 657, 489, 569, 857, 168, 603, 742, 563, 339, 796, 99, 57, 53, 38, 41, 77, 605, 686, 522, 521, 814, 818, 721, 737, 734, 745, 874, 747]
🧮 Total tiendas: 951
📌 Total claves en df_tiendas: 727
📌 Total claves en df_demog: 5468
✅ Tiendas con datos demográficos: 951
🚫 Tiendas sin datos demográficos: 36
⚠️ Claves mal formadas: 35
⚠️ Tiendas con valores distintos en columnas originales: 0
✅ 'tiendas_ageb_demografia.csv' generado correctamente.
📊 Ventas únicas (TIENDA_ID): 948
📦 Tiendas agregadas en df_grouped: 951
✅ 'tiendas_ageb_demografia_ventas.csv' generado correctamente.
